## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# 1. BUSINESS UNDERSTANDING 
## Business Problem

The entertainment industry is undergoing rapid transformation, with original content emerging as a key driver of audience engagement and revenue growth. Major players like Netflix, Amazon, and other sources are heavily investing in original films —reaping substantial financial returns and strengthening their brand presence.
Recognizing this trend, Flix company has made the strategic decision to launch a new movie studio. However, it currently lacks the data-driven insights necessary to understand what factors contribute to a film’s box office success.
As data scientists, our role is to explore publicly available movie performance data to uncover patterns that indicates
what makes a movie financially successful. The goal is to provide clear data- driven, actionable recommendations that will help guide decisions about genre, budget size, release timing, and other production choices.



## Stakeholders and Use Cases
Primary Stakeholder:
Head of the New Movie Studio

Use Case:
Leverage data-driven insights to inform strategic decisions on film production. This includes identifying high-performing genres, determining optimal budget ranges, selecting ideal release windows, and shaping casting strategies—all aimed at maximizing box office success and return on investment.

## Project Objectives
* Identify which genres perform best at the box office, considering revenue and profitability.

* Analyze the impact of budget, runtime, cast, and release month on a film’s success.

* Provide actionable recommendations for the types of films the company should produce.

## Conclusion: Implications and Recommendations

This project provides a data-driven foundation to support the successful launch of Flix company’s new movie studio while reducing financial risk. By uncovering the key factors that correlate with box office success, the Head of Studio is equipped to make informed, strategic decisions, including:

* Genre Selection: Focus on genres with a strong track record of performance.

* Budget Planning: Allocate production budgets based on historically successful investment ranges.

* Release Strategy: Optimize release timing to align with peak audience engagement periods.

* Talent Strategy: Identify the cast and crew characteristics commonly linked to high-grossing films.



# 2. DATA UNDERSTANDING

## Data Sources

This project uses data from three high-quality, complementary sources of movie data:

### 1. Box Office Mojo (bom.movie_gross.csv.gz)

Provides domestic box office revenue data.

Includes key features such as: title, studio, domestic_gross, release_date, and year.

used to determine the financial performance of films.

 ### 2. IMDb(Internet Movie Database) (im.db.zip)

Contains detailed metadata about films and user-ratings.

Key tables used:

* movie_basics: Includes primary_title, original_title, genres, runtime_minutes, and start_year.

* movie_ratings: Contains user rating data (average_rating, num_votes).

used for movie characteristics and audience quality perceptions.

 ### 3. TheMovieDB (TMDb) (tmdb.movies.csv.gz)

TheMovieDb entails the following:
* User-generated popularity 
* voting data

Key features: title, popularity, vote_average, vote_count, release_date, genres, budget, revenue

Purpose: Complements Box Office Mojo and IMDb with:

Popularity metrics: Show which films gain audience traction pre- and post-release

Vote data: Allows cross-comparison with IMDb ratings


### 4. The Numbers (tn.movie_budgets.csv.gz)

It consits of  Film production budgets and worldwide gross

Key Features:

* Release_date, movie, production_budget, domestic_gross, worldwide_gross

Why It Matters:

* Gives a complete financial picture by providing both the cost of making the film (production budget) and revenue generated globally.

* Allows calculation of Return on Investment (ROI) — one of the most important metrics when deciding which types of films to produce.


# 3. DATA PREPARATION

To determine which types of films perform best at the box office, it is essential to integrate and clean datasets from Box Office Mojo, IMDb, and The Movie Database (TMDb). The following section outlines the data preparation process, including code examples and explanations for each step.

## Loading and inspecting the Raw data 

Befoe we load our datasets , we will import libraries which we will use.


In [2]:
# Importing necessary libraries
import pandas as pd 
import sqlite3
import zipfile


The next step would be to load our datasets and see what it entails .

In [3]:
# loading the dataset of Box Office Mojo
box_office_mojo= pd.read_csv('zippedData/bom.movie_gross.csv/bom.movie_gross.csv')
# displays the first few rows 
box_office_mojo.head()


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


The dataset contains the domestic office box revenue which we will use as the target variable.  
We'll filter for movies only (not re-releases or limited runs), and ensure all rows have valid gross data.

In [4]:
# Loading the sqlite database of IMDB
# Connecting to sqlite IMDB database
# Step 1: Extract the .db file from the zip archive
# with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
#     zip_ref.extractall('zippedData')  # This will extract im.db into zippedData/

# Step 2: Connect to the extracted SQLite database
conn = sqlite3.connect('zippedData/im.db')

# Step 3: Load tables into pandas DataFrames
# basics = pd.read_sql('SELECT * FROM movie_basics', conn)
# ratings = pd.read_sql('SELECT * FROM movie_ratings', conn)
 # Close connection
# conn.close()

The IMDB database consists of :
movie_basics: includes metadata like genres, runtime_minutes, and start_year.

movie_ratings: includes average_rating and num_votes.

These are used to describe each film and estimate perceived quality and popularity.

In [5]:
# Loading TheMovieDB
movie_Db= pd.read_csv('zippedData/tmdb.movies.csv/tmdb.movies.csv',index_col=0 )
# Views the first few rows
movie_Db.head()
 #movie_Db.info()
# movie_Db.shape

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


TheMovieDb adds user interest indicators (popularity, vote_average, vote_count) and financial info (budget, revenue) not found in the other sources.

In [6]:
# Load movie budget dataset (TSV format, gzip-compressed)
movie_budget = pd.read_csv('zippedData/tn.movie_budgets.csv/tn.movie_budgets.csv') #sep='\t')

# Display the first few rows
movie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## DATA CLEANING
### 1. BOX OFFICE DATASET

In this dataset, we will drop the column that we intend not to use which is the the foreign gross column. We intend to find the the financial performance of the movies in this dataset. The methods used to clean this dataset are;(.info(),.describe(),.drop(),.fillna(),.dropna(),.isna(),.duplicates()).
This dataset has a total of 3387 rows and 5 columns.
It is essential to fill the numerical columns with mean so as to avoid outliers and this will inturn help us generate insights that are not missleading.

In [7]:
#Summary of our data structure
box_office_mojo.info()
box_office_mojo.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


(3387, 5)

In [8]:
# Summary statics of the dataframe
box_office_mojo['domestic_gross'].describe()

count    3.359000e+03
mean     2.874585e+07
std      6.698250e+07
min      1.000000e+02
25%      1.200000e+05
50%      1.400000e+06
75%      2.790000e+07
max      9.367000e+08
Name: domestic_gross, dtype: float64

To avoid droping columns from the main dataframe, renaming the dataframe is very important.

In [9]:
#drop the foreign_gross
refined_box = box_office_mojo.drop(columns= ['foreign_gross'])

Filling domestic gross and foreign gross with median also ensures that there are no nulls in the domestic_gross column, which is essential for: plotting (histograms and scatter plots) and Running statistical tests and regression models.

In [10]:
#filling domestic gross with median (numerical column),
refined_box['domestic_gross'].fillna(refined_box['domestic_gross'].median(), inplace=True)

#checking for null values
refined_box.isna().sum()

title             0
studio            5
domestic_gross    0
year              0
dtype: int64

In [11]:
# drop the missing values
refined_box.dropna()

,title,studio,domestic_gross,year
0,Toy Story 3,BV,415000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,2010
3,Inception,WB,292600000.0,2010
4,Shrek Forever After,P/DW,238700000.0,2010
...,...,...,...,...
3382,The Quake,Magn.,6200.0,2018
3383,Edward II (2018 re-release),FM,4800.0,2018
3384,El Pacto,Sony,2500.0,2018
3385,The Swan,Synergetic,2400.0,2018


In [12]:
#Filling the null values in the studio column with unknown
refined_box = refined_box.fillna('Unknown')

In [13]:
#Checking if there are any missing values remaining
refined_box.isna().sum()

title             0
studio            0
domestic_gross    0
year              0
dtype: int64

In [14]:
#checking for duplicated rows in the dataset
refined_box.duplicated().value_counts()  

False    3387
dtype: int64

### 2. THE MOVIE DB

In this dataset, we will drop the column that we intend not to use which is the the original_language column. We intend to Show which films gain audience traction pre- and post-release. The methods used to clean this dataset are;(.info(),.describe(),.drop(),.isna(),.duplicates())
This dataset has a total of 26517 rows and 9 columns.

In [15]:
#Summary of our data structure
movie_Db.info ()
movie_Db.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


(26517, 9)

In [16]:
#Describes the summary statistics of the dataset
movie_Db.describe()

,id,popularity,vote_average,vote_count
count,26517.000000,26517.000000,26517.000000,26517.000000
mean,295050.153260,3.130912,5.991281,194.224837
std,153661.615648,4.355229,1.852946,960.961095
min,27.000000,0.600000,0.000000,1.000000
25%,157851.000000,0.600000,5.000000,2.000000
50%,309581.000000,1.374000,6.000000,5.000000
75%,419542.000000,3.694000,7.000000,28.000000
max,608444.000000,80.773000,10.000000,22186.000000


In [17]:
#Droping columns
cleaned_moviedb = movie_Db.drop(columns=["original_language"])
#cleaned_moviedb.head()

In [18]:
#checking for null values
cleaned_moviedb.isna().sum()

genre_ids         0
id                0
original_title    0
popularity        0
release_date      0
title             0
vote_average      0
vote_count        0
dtype: int64

Make use of the pd.to_datetime method to change the datatype from object to datetime which is makes it usable in the analysis  stage. It basically sets the data up for comparison in relation to time.
errors = 'coerce' tells pandas to convert the data properly.

In [19]:
#Converting dates
cleaned_moviedb['release_date'] = pd.to_datetime(cleaned_moviedb['release_date'], errors='coerce')

In [20]:
#Checks for duplicates 
cleaned_moviedb.duplicated(subset='id').value_counts()

False    25497
True      1020
dtype: int64

In [21]:
#drop the dulicates
cleaned_moviedb.drop_duplicates(inplace = True)

### 3. The Numbers Dataset (`tn.movie_budgets.csv.gz`)

To prepare the *The Numbers* dataset for analysis, we performed the following cleaning steps:

### 1. Loading the dataset
The dataset was read using `pandas.read_csv()`.


In [36]:


budgets_df = pd.read_csv("zippedData/tn.movie_budgets.csv/tn.movie_budgets.csv")
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


### 2. Inspecting Columns and Data Types

In [28]:
budgets_df.info()
budgets_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

### 3. Cleaning Column Names

In [29]:
# Make column names lowercase and replace spaces with underscores
budgets_df.columns = budgets_df.columns.str.lower().str.replace(' ', '_')

### 4. Converting Currency Columns to Numeric

In [30]:
currency_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

for col in currency_cols:
    budgets_df[col] = (
        budgets_df[col]
        .replace('[\$,]', '', regex=True)
        .astype(float)
    )

### 5. Converting Dates to Datetime Format

In [31]:
# Convert release_date to datetime
budgets_df['release_date'] = pd.to_datetime(budgets_df['release_date'])

# Create a new column for release year
budgets_df['release_year'] = budgets_df['release_date'].dt.year

### 6. Checking for Missing Values

In [32]:
budgets_df.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
release_year         0
dtype: int64

In [33]:
# Drop rows where essential data is missing
budgets_df = budgets_df.dropna(subset=['movie', 'production_budget', 'worldwide_gross'])

### 7. Creating a New Profit Column

In [34]:
budgets_df['profit'] = budgets_df['worldwide_gross'] - budgets_df['production_budget']

### 8. Final Preview

In [35]:
budgets_df[['movie', 'release_year', 'production_budget', 'worldwide_gross', 'profit']].head()

,movie,release_year,production_budget,worldwide_gross,profit
0,Avatar,2009,425000000.0,2.776345e+09,2.351345e+09
1,Pirates of the Caribbean: On Stranger Tides,2011,410600000.0,1.045664e+09,6.350639e+08
2,Dark Phoenix,2019,350000000.0,1.497624e+08,-2.002376e+08
3,Avengers: Age of Ultron,2015,330600000.0,1.403014e+09,1.072414e+09
4,Star Wars Ep. VIII: The Last Jedi,2017,317000000.0,1.316722e+09,9.997217e+08
